In [1]:
import tensorflow as tf
import BHDVCS 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
data = pd.read_csv('data.csv')

reh = np.array(data['ReH']).transpose()
ree = np.array(data['ReE']).transpose()
reht = np.array(data['ReHtilde']).transpose()

cff = np.concatenate((reh, ree, reht), axis=0)

g = 14
constants = np.array(data['dvcs'])[0:36*g+36]

phi = np.array(data['phi_x'])[0:36*g+36]
F1 = np.array(data['F1'])[0:36*g+36]
F2 = np.array(data['F2'])[0:36*g+36]

q2 = np.array(data['QQ'])[0:36*g+36]
xb = np.array(data['x_b'])[0:36*g+36]
k = np.array(data['k'])[0:36*g+36]
t = np.array(data['t'])[0:36*g+36]

F = np.array(data['F'])[0:36*g+36]
errF = np.array(data['errF'])[0:36*g+36]

data = data.drop(columns=['#Set','index','F','errF','ReH','ReE','ReHtilde'])

cols = ['phi_x', 'QQ', 'x_b', 't', 'k', 'F1', 'F2', 'dvcs']

data = data.reindex(columns=cols)

In [5]:
func = BHDVCS.BHDVCS()
def los(y_true, y_pred):
    
    try:
        a = np.where(reh == y_true[0].numpy())
        b = np.where(ree == y_true[1].numpy())
        c = np.where(reht == y_true[2].numpy())
        g = (a and b and c)[0][0]+1
    except:
        g = 0
    
    dat = data[g:g+1].to_numpy().transpose()
    
    
    
    j = 0
    tot = 0
    for i in range(0,360,10):
        dat[0] = i
        tot += abs(func.TotalUUXS_curve_fit(dat, y_true[0], y_true[1], y_true[2])-func.TotalUUXS_curve_fit(dat, y_pred[0], y_pred[1], y_pred[2]))
        j+=1
    
    return tot*tot


In [6]:
x_data = pd.read_csv('data.csv')
y_data = pd.read_csv('data.csv')
#,'dvcs'
#k,QQ,x_b,t,phi_x,F,errF,F1,F2,dvcs
x_data = x_data.drop(columns=['#Set','index','phi_x','errF','ReH','ReE','ReHtilde'])
y_data = y_data.drop(columns=['#Set','index','k','QQ','x_b','t','phi_x','F','errF','F1','F2','dvcs'])

x = pd.DataFrame(x_data.loc[0:36*g+36:36]).to_numpy()
y = pd.DataFrame(y_data.loc[0:36*g+36:36]).to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

iters = x_test.ndim

x_train, x_test, y_train,y_test = x_train / 255.0, x_test / 255.0,y_train / 255.0, y_test / 255.0


In [7]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(8,)))

model.add(tf.keras.layers.Dense(16, activation='tanh'))
model.add(tf.keras.layers.Dense(32, activation='tanh'))

model.add(tf.keras.layers.Dense(3))


model.compile(optimizer='adam', loss=los)
history = model.fit(x_train, y_train, epochs=1500)



arr = model.predict(x_test)*255.0


Epoch 1/1500
1/1 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 2/1500
1/1 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 3/1500
1/1 [==============================] - 0s 996us/step - loss: 0.0034
Epoch 4/1500
1/1 [==============================] - 0s 2ms/step - loss: 0.0023
Epoch 5/1500
1/1 [==============================] - 0s 1000us/step - loss: 0.0028
Epoch 6/1500
1/1 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 7/1500
1/1 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 8/1500
1/1 [==============================] - 0s 1ms/step - loss: 0.0017
Epoch 9/1500
1/1 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 10/1500
1/1 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 11/1500
1/1 [==============================] - 0s 1ms/step - loss: 6.8779e-04
Epoch 12/1500
1/1 [==============================] - 0s 1ms/step - loss: 1.5560e-04
Epoch 13/1500
1/1 [=================

In [8]:
for i in range(0, iters):

    data = pd.read_csv('data.csv')
    a = np.where(reh == y_test[i][0]*255)
    b = np.where(ree == y_test[i][1]*255)
    c = np.where(reht == y_test[i][2]*255)
    g = (a and b and c)[0][0] 

    constants = np.array(data['dvcs'])[g:g+36]

    phi = np.array(data['phi_x'])[g:g+36]
    F1 = np.array(data['F1'])[g:g+36]
    F2 = np.array(data['F2'])[g:g+36]

    q2 = np.array(data['QQ'])[g:g+36]
    xb = np.array(data['x_b'])[g:g+36]
    k = np.array(data['k'])[g:g+36]
    t = np.array(data['t'])[g:g+36]

    F = np.array(data['F'])[g:g+36]
    errF = np.array(data['errF'])[g:g+36]


    func = BHDVCS.BHDVCS()

    data1 = (phi, q2, xb, t, k, F1, F2, constants)

    params_fit, pcov = curve_fit(func.TotalUUXS_curve_fit, data1, F, sigma=errF)
    
    
    print("Errors Set ", i +1,":" )
    reh_chi = (params_fit[0] - y_test[i][0]*255)/ (y_test[i][0]*255)
    ree_chi = (params_fit[1] - y_test[i][1]*255)/ (y_test[i][1]*255)
    reht_chi = (params_fit[2] - y_test[i][2]*255)/ (y_test[i][2]*255)
    
    print("Chi  ReH", reh_chi, "ReE", ree_chi, "ReHT", reht_chi)
    
    
    reh_ml = (arr[i][0] - y_test[i][0]*255)/ (y_test[i][0]*255)
    ree_ml = (arr[i][1] - y_test[i][1]*255)/ (y_test[i][1]*255)
    reht_ml = (arr[i][2] - y_test[i][2]*255)/ (y_test[i][2]*255)
    print("ML  ReH", reh_ml, "ReE", ree_ml, "ReHT", reht_ml)
    

Errors Set  1 :
Chi  ReH -0.133712546061349 ReE -0.17175076477240778 ReHT 0.06574362138474582
ML  ReH -0.3696270113613275 ReE -0.07919397653843191 ReHT -0.3719670186132768
Errors Set  2 :
Chi  ReH -0.09459516432642315 ReE -0.08530219227052974 ReHT 0.11741215571979306
ML  ReH 0.321180254135486 ReE 0.04650482032074683 ReHT 0.3669184941112781
